In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


## Preprocessing:-

In [0]:
import helper
import numpy as np


In [3]:
#Reading the Script
with open("got1.txt",'r') as file:
  got1=file.read()
with open("got2.txt",'r') as file:
  got2=file.read()
with open("got3.txt",'r') as file:
  got3=file.read()
with open("got1.txt",'r') as file:
  got4=file.read()
with open("got1.txt",'r') as file:
  got5=file.read()

text = got1+got2+got3+got4+got5
print(len(text))
print(len(got1))

9702721
1733704


In [4]:
view_sentence_range = (0, 10)

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 56308
Number of scenes: 42815
Average number of sentences in each scene: 0.0017984351278757444
Number of lines: 42892
Average number of words in each line: 41.90688240231279

The sentences 0 to 10:

The comet’s tail spread across the dawn, a red slash that bled above the crags of Dragonstone like a wound in the pink and purple sky.

The maester stood on the windswept balcony outside his chambers. It was here the ravens came, after long flight. Their droppings speckled the gargoyles that rose twelve feet tall on either side of him, a hellhound and a wyvern, two of the thousand that brooded over the walls of the ancient fortress. When first he came to Dragonstone, the army of stone grotesques had made him uneasy, but as the years passed he had grown used to them. Now he thought of them as old friends. The three of them watched the sky together with foreboding.

The maester did not believe in omens. And yet . . . old as he was, Cressen had

In [0]:

def create_lookup_tables(text):
      #text is split into words
      words = sorted(list(set(text)))
      vocab_to_int = {word:index for index,word in enumerate(words)}
      int_to_vocab = {index:word for index,word in enumerate(words)}
      return vocab_to_int,int_to_vocab


In [0]:
def token_lookup():
    mark_to_token={}
    mark_to_token['.'] = "||Period||"
    mark_to_token[','] = "||Comma||"
    mark_to_token['"'] = "||QuotationMark||"
    mark_to_token[';'] = "||Semicolon||"
    mark_to_token['!'] = "||Exclamationmark||"
    mark_to_token['?'] = "||Questionmark||"
    mark_to_token['('] = "||LeftParentheses||"
    mark_to_token[')'] = "||RightParentheses||"
    mark_to_token['--'] = "||Dash||"
    mark_to_token['\n'] = "||Return||"
    return mark_to_token


In [0]:
def preprocess_and_save_data(dataset_path, token_lookup, create_lookup_tables):
    import pickle
    """
    Preprocess Text Data
    """
    with open("got1.txt",'r') as file:
      got1=file.read()
    with open("got2.txt",'r') as file:
      got2=file.read()
    with open("got3.txt",'r') as file:
      got3=file.read()
    with open("got1.txt",'r') as file:
      got4=file.read()
    with open("got1.txt",'r') as file:
      got5=file.read()

    text = got1+got2+got3+got4+got5
    

    token_dict = token_lookup()
    for key, token in token_dict.items():
        text = text.replace(key, ' {} '.format(token))

    text = text.lower()
    text = text.split()

    vocab_to_int, int_to_vocab = create_lookup_tables(text)
    int_text = [vocab_to_int[word] for word in text]
    pickle.dump((int_text, vocab_to_int, int_to_vocab, token_dict), open('preprocess.p', 'wb'))



In [0]:
preprocess_and_save_data("none",token_lookup,create_lookup_tables)


In [9]:
text[:100]

'\nThe comet’s tail spread across the dawn, a red slash that bled above the crags of Dragonstone like '

**Now our checkpoint has been created**

# Building The Netwok

In [0]:
import pickle

int_text, vocab_to_int, int_to_vocab, token_dict = pickle.load(open('preprocess.p', mode='rb'))

In [0]:
import tensorflow as tf
import numpy as np
import random

In [0]:
def get_inputs():
  Inputs = tf.placeholder(tf.int32,[None,None],name='input')
  Targets = tf.placeholder(tf.int32,[None,None],name='Targets')
  LearningRate= tf.placeholder(tf.float32,name='learningrate')
  return Inputs,Targets,LearningRate

In [0]:
def get_init_cell(batch_size,rnn_size):
  def getCell(rnn_siz):
    rnn_cell = tf.contrib.rnn.BasicLSTMCell(rnn_siz)
    drop = tf.contrib.rnn.DropoutWrapper(rnn_cell,output_keep_prob=0.90)
    return drop

  cell = tf.contrib.rnn.MultiRNNCell([getCell(rnn_size[i]) for i in range(2)])
  initial_state = cell.zero_state(batch_size,tf.float32)
  initial_state =  tf.identity(initial_state,"initial_state")
  return cell,initial_state


In [0]:
def get_embed(input_data,vocab_size,embed_dim):
  embeddings = tf.Variable(tf.truncated_normal([vocab_size,embed_dim]))
  embed = tf.nn.embedding_lookup(embeddings,input_data)
  return embed


In [0]:
def build_rnn(cell,inputs):
  outputs,final_state  = tf.nn.dynamic_rnn(cell,inputs,dtype=tf.float32)
  final_state = tf.identity(final_state,name='final_state')

  return (outputs,final_state)


In [0]:
def build_nn(cell,rnn_size,input_data,vocab_size,embed_dim):
  embed = get_embed(input_data,vocab_size,embed_dim)
  (outputs,final_state)  = build_rnn(cell,embed)

  Logits = tf.contrib.layers.fully_connected(outputs,vocab_size,activation_fn=tf.keras.activations.linear)

  return (Logits,final_state)


In [0]:
def get_batches(int_text,batch_size,seq_len):
  chars_per_batch = batch_size*seq_len
  num_batches = len(int_text)//chars_per_batch
  int_text = int_text[:num_batches*chars_per_batch]
  batches = np.zeros([num_batches,2,batch_size,seq_len])
  counter = 0
  for i in range(batch_size):
      for j in range(0,num_batches):
            batches[j,0,i,:] = int_text[counter:counter+seq_len]
            y_temp = int_text[counter+1:counter+seq_len+1]
            batches[j,1,i,:len(y_temp)] = y_temp            
            counter+=seq_len
  batches[num_batches-1,1,-1,-1] = int_text[0]
  return batches            

In [0]:
#Hyperparameters

num_epochs = 200
batch_size=  256
rnn_size = [300,300]
embed_dims = 350
seq_len = 42
learning_rate = 0.001
show_every_n_batches = 50

save_dir = './save'



In [0]:
#Building the Graph
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
  vocab_size = len(int_to_vocab)
  input_text,targets,lr = get_inputs()
  input_data_shape = tf.shape(input_text)
  cell,initial_state = get_init_cell(input_data_shape[0],rnn_size)
  Logits,Final_state =   build_nn(cell,rnn_size,input_text,vocab_size,embed_dims)

  probs = tf.nn.softmax(Logits,name='probs')
  #Loss Function
  cost = seq2seq.sequence_loss(Logits,targets,
                               tf.ones([input_data_shape[0],input_data_shape[1]]))
  
  #optimizer
  optimizer = tf.train.AdamOptimizer(lr)

  gradients=optimizer.compute_gradients(cost)

  clipped_gradients = [(tf.clip_by_value(grad,-1.,1.),var) for grad,var in gradients if grad is not None]

  train_op = optimizer.apply_gradients(clipped_gradients)


In [0]:
#Training the Network
batches = get_batches(int_text,batch_size,seq_len)

with tf.Session(graph=train_graph) as sess:
  sess.run(tf.global_variables_initializer())

  for epoch_i in range(num_epochs):
      state = sess.run(initial_state,{input_text: batches[0][0]})
      for batch_i,(x,y) in enumerate(batches):
           feed = {input_text:x,
                  targets:y,
                  lr:learning_rate,
                  initial_state:state}

           train_loss,state,_ = sess.run([cost,Final_state,train_op],feed_dict=feed)
           if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))
      if(epoch_i%5==0):
            # Save Model
            saver = tf.train.Saver()
            saver.save(sess, save_dir)
            print('Model Trained and Saved')

  # Save Model
  saver = tf.train.Saver()
  saver.save(sess, save_dir)
  print('Model Trained and Saved')


In [0]:
pickle.dump((seq_len,save_dir), open('params.p', 'wb'))

In [0]:
def get_tensors(loaded_graph):
  with tf.Session(graph=loaded_graph):
      return (tf.get_default_graph().get_tensor_by_name('input:0')
      ,tf.get_default_graph().get_tensor_by_name("initial_state:0")
      ,tf.get_default_graph().get_tensor_by_name('final_state:0')
      ,tf.get_default_graph().get_tensor_by_name('probs:0') )

In [0]:
def pick_word(probabilities,int_to_vocab,top_n=5):
      p  = np.squeeze(probabilities)
      vocab_size = len(int_to_vocab)
      p[np.argsort(p)[:-top_n]] = 0
      p = p / np.sum(p)
      c = np.random.choice(vocab_size, 1, p=p)[0]
      return int_to_vocab[c]

In [36]:
seq_length = 42

prime_word = 'jon'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
  loader = tf.train.import_meta_graph('save' + '.meta')
  loader.restore(sess, save_dir)

  input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

  gen_sentences = [prime_word]
  prev_state = sess.run(initial_state,{input_text:np.array([[1]])})

  for n in range(gen_length):
    #Dynamic Input
        dyn_input = [ [vocab_to_int[word] for word in gen_sentences[-seq_length:]] ]
        dyn_seq_length = len(dyn_input[0])
        probabilities, prev_state = sess.run(
                [probs, final_state],
                {input_text: dyn_input, initial_state: prev_state})
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)
        gen_sentences.append(pred_word)
  
  tv_script = ' '.join(gen_sentences)
  for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
  tv_script = tv_script.replace('\n ', '\n')
  tv_script = tv_script.replace('( ', '(')
        
  print(tv_script)



INFO:tensorflow:Restoring parameters from ./save
jon arryn resumed us. ”

a different sort of pain they made a rude noise. the girl was a timid older than her, arya thought. joffrey made a jest, to keep her in the mountains. i cannot fault them to her. she could feel the blood of the poor sword of yours. but it was the same stories long ago, with the way his head seemed to require a certain show of his... and the puissant death. ”

“i know the same. ” the ruby on the throat was hot to make a certain sort of natural that look like this. she glanced up in the saddle. “i am not so scared. you are the blood of your youth, my lady. ”

the boy, he thought. the words was growing harder to her, and tyrion saw her, with all the chivalry of its end. in a growing distance, a lordling bellows and a direwolf in silence.

the first line passed to lay a long summer mile,
